In [1]:
import Base.size

type Vertex
    id::Int
    edgesIn::Array
    edgesOut::Array
    flow::Int
end

degree(v::Vertex) = length(v.edgesIn) + length(v.edgesOut)
outdegree(v::Vertex) = length(v.edgesOut)
indegree(v::Vertex) = length(v.edgesIn)
flow(v::Vertex) = begin
    m = v.flow
    if !isempty(v.edgesIn)
        m += sum([e.flow for e in v.edgesIn])
    end
    if !isempty(v.edgesOut)
        m -= sum([e.flow for e in v.edgesOut])
    end
    m
end

Base.show(io::IO, v::Vertex) = print(io,"$(v.id)")

type Edge
    tail::Vertex
    head::Vertex
    cap::Int
    low::Int
    cost::Int
    flow::Int
end

Base.show(io::IO, e::Edge) = print(io,"($(e.tail))-[$(e.flow),$(e.cap),$(e.cost)]->($(e.head))")

type Graph
    vertices::Array{Vertex,1}
    edges::Array{Edge,1}
end

order(g::Graph) = length(g.vertices)
size(g::Graph) = length(g.edges)

Base.show(io::IO, g::Graph) = begin
    println(io, "G(V,E) = [$(order(g)),$(size(g))]")
    for i = 1:size(g)
        if size(g) <= 50 || i <= 10 || i > (size(g)-10)
            e = g.edges[i]
            println(io, "$i: $e")
        end
        if size(g) > 50 && i == 11
            println(io, "... $(size(g) - 20) ...")
        end
    end
end

Vertex(id::Int; flow=0) = Vertex(id, Edge[], Edge[], flow)
Edge(t::Vertex, h::Vertex; cap=0,low=0,cost=0,flow=0) = begin
    e = Edge(t,h,cap,low,cost,flow)
    push!(t.edgesOut, e)
    push!(h.edgesIn, e)
    e
end

Edge

In [2]:
function g4()
    s = Vertex(1, flow=30)
    u = Vertex(2)
    v = Vertex(3)
    t = Vertex(4, flow=-30)
    vrts = Vertex[s, u, v, t]
    
    e1 = Edge(s, u, cap=20)
    e2 = Edge(s, v, cap=10)
    e3 = Edge(u, t, cap=10)
    e4 = Edge(v, t, cap=20)
    e5 = Edge(u, v, cap=30)
    edgs = Edge[e1, e2, e3, e4, e5]
    
    Graph(vrts, edgs)
end

g4 (generic function with 1 method)

In [3]:
g = g4()

G(V,E) = [4,5]
1: (1)-[0,20,0]->(2)
2: (1)-[0,10,0]->(3)
3: (2)-[0,10,0]->(4)
4: (3)-[0,20,0]->(4)
5: (2)-[0,30,0]->(3)


In [4]:
g.vertices

4-element Array{Vertex,1}:
 1
 2
 3
 4

In [5]:
for v in g.vertices
    println("$(v.id): degree $(degree(v)), in $(indegree(v)), out $(outdegree(v))")
end

1: degree 2, in 0, out 2


In [6]:
g.edges

5-element Array{Edge,1}:
 (1)-[0,20,0]->(2)
 (1)-[0,10,0]->(3)
 (2)-[0,10,0]->(4)
 (3)-[0,20,0]->(4)
 (2)-[0,30,0]->(3)

In [7]:
function netg(file::AbstractString)
    open(file) do f
        vertices = nothing
        edges = nothing
        k = 0
        
        xint(n::Int, line::AbstractString, k::AbstractString, v::AbstractString) = try
            parse(Int, v)
        catch e
            error("$n, $line - $k: $(e.msg)")
        end
    
        for (n, line) in enumerate(eachline(f))
            xint(k::AbstractString, v::AbstractString) = xint(n, line, k, v)
            xerror(m::AbstractString) = error("$n, $line - $m")
            
            if startswith(line, 'c')
                continue
            elseif startswith(line, 'p')
                m = split(line, ' ')
                length(m) == 4 || xerror("Problem line (4): $(length(m))") 
                order = xint("Number of vertices", m[3])
                size = xint("Number of edges", m[4])
                
                vertices = Array(Vertex, order)
                for i = 1:order
                    vertices[i] = Vertex(i)
                end
                edges = Array(Edge, size)
            elseif startswith(line, 'n')
                m = split(line, ' ')
                length(m) == 3 || xerror("Vertex line (3): $(length(m))")
                id = xint("Vertex id", m[2])
                flow = xint("Vertex flow", m[3])
                
                vertices[id].flow = flow
            elseif startswith(line, 'a')
                k < length(edges) || xerror("Edge extra: $(length(edges))")
                m = split(line, ' ')
                length(m) == 6 || xerror("Edge line (6): $(length(m))")
                tail_idx = xint("Edge tail", m[2])
                head_idx = xint("Edge head", m[3])
                low = xint("Edge lower bound", m[4])
                cap = xint("Edge capacity", m[5])
                cost = xint("Edge cost", m[6])
                
                tail = vertices[tail_idx]
                head = vertices[head_idx]
                edges[k+=1] = Edge(tail, head, cap=cap, low=low, cost=cost)
            else
                xerror("Unknown line")
            end
        end
        k == length(edges) || error("Missing edges: $k of $(length(edges)) ($(length(edges) - k))")
        
        Graph(vertices, edges)
    end
end

2: degree 3, in 1, out 2
3: degree 3, in 2, out 1
4: degree 2, in 2, out 0


netg (generic function with 1 method)

In [8]:
netg("../data/big8.net")

G(V,E) = [50000,500000]
1: (1)-[0,1163,26]->(34671)
2: (1)-[0,41753,377]->(35165)
3: (1)-[0,12534,456]->(25991)
4: (1)-[0,12078,292]->(7613)
5: (1)-[0,42459,758]->(41134)
6: (1)-[0,40063,351]->(17673)
7: (1)-[0,18964,705]->(2942)
8: (712)-[0,1163,651]->(14463)
9: (712)-[0,29636,25]->(47696)
10: (712)-[0,38698,971]->(29864)
... 499980 ...
499991: (48234)-[0,41967,179]->(4151)
499992: (48579)-[0,795,540]->(49981)
499993: (48579)-[0,795,106]->(22932)
499994: (48579)-[0,49604,553]->(45285)
499995: (49478)-[0,795,28]->(14335)
499996: (49478)-[0,14567,108]->(18969)
499997: (49478)-[0,32703,176]->(39022)
499998: (49478)-[0,27171,601]->(7800)
499999: (49478)-[0,1586,902]->(26459)
500000: (49478)-[0,27081,228]->(22158)


In [9]:
function st(g::Graph)
    s = nothing
    t = nothing
    n = 0
    for v in g.vertices
        if v.flow > 0
            s = v
            n += 1
        elseif v.flow < 0
            t = v
            n += 1
        end
    end
    isa(s, Vertex) || error("Source not found")
    isa(t, Vertex) || error("Target not found")
    n == 2 || error("Not ST")
    indegree(s) == 0 || error("Souce invalid")
    outdegree(t) == 0 || error("Target invalid")
    s, t
end

st (generic function with 1 method)

In [10]:
using DataStructures

type PathNode
    parent::PathNode
    length::Int
    reverse::Bool
    edge::Edge
    
    PathNode(e::Edge,r=false) = (p = new(); (p.length, p.reverse, p.edge) = (0, r, e); p)
    PathNode(p::PathNode, r::Bool, e::Edge) = new(p, p.length + 1, r, e)
end

head(p::PathNode) = p.reverse ? p.edge.tail : p.edge.head
tail(p::PathNode) = p.reverse ? p.edge.head : p.edge.tail
flow(p::PathNode) = p.reverse ? (p.edge.cap - p.edge.flow) : p.edge.flow
free(p::PathNode) = p.reverse ? p.edge.flow : (p.edge.cap - p.edge.flow)
cap(p::PathNode) = p.edge.cap
cost(p::PathNode) = p.reverse ? -p.edge.cost : p.edge.cost

Base.show(io::IO, p::PathNode) = print(io,"($(tail(p)))-[$(flow(p)),$(cap(p)),$(cost(p))]->($(head(p))), reverse=$(p.reverse)")

path(leaf::PathNode) = begin
    p = Array(PathNode, leaf.length)
    i = leaf
    while isdefined(i, :parent)
        p[i.length] = PathNode(i.edge, i.reverse) # inverse root distance
        i = i.parent
    end
    p
end

function availableflow(p::Array{PathNode,1})
    minimum([free(i) for i in p])
end

function residualpath(g::Graph, s::Vertex, t::Vertex)
    flow(s) > 0 || return nothing
    nodes = Queue(PathNode)
    mark = falses(order(g))
    
    enqueue!(nodes, PathNode(Edge(Vertex(0),s,0,0,0,0)))
    mark[s.id] = true
    
    while length(nodes) > 0
        n = dequeue!(nodes)
        v = head(n)
        for e in v.edgesOut
            if e.flow == e.cap || mark[e.head.id]
                continue
            end
            next = PathNode(n,false,e)
            if is(e.head, t)
                return path(next)
            end
            enqueue!(nodes, next)
            mark[e.head.id] = true
        end
        for e in v.edgesIn
            if e.flow == 0 || mark[e.tail.id]
                continue
            end
            next = PathNode(n,true,e)
            if is(e.tail, t)
                return path(next)
            end
            enqueue!(nodes, next)
            mark[e.tail.id] = true
        end
    end
    nothing
end

residualpath (generic function with 1 method)

In [11]:
s, t = st(g)
p = residualpath(g, s, t)

2-element Array{PathNode,1}:
 (1)-[0,20,0]->(2), reverse=false
 (2)-[0,10,0]->(4), reverse=false

In [12]:
function residualflow(p::Array{PathNode,1})
    v = tail(p[1])
    min(flow(v), availableflow(p))
end

residualflow (generic function with 1 method)

In [13]:
delta = residualflow(p)

10

In [14]:
function updateflow!(path::Array{PathNode,1}, delta::Int)
    for p in path
        p.edge.flow += p.reverse ? -delta : delta
    end
    nothing
end

updateflow! (generic function with 1 method)

In [15]:
updateflow!(p, delta)
g

G(V,E) = [4,5]
1: (1)-[10,20,0]->(2)
2: (1)-[0,10,0]->(3)
3: (2)-[10,10,0]->(4)
4: (3)-[0,20,0]->(4)
5: (2)-[0,30,0]->(3)


In [16]:
# Reference
# Algorithms Design. Tardos and Kleinberg
# 7.1 The Maximum-Flow Problem and the Ford-Fulkerson Algorithm, pp. 338-346
function maxflow!(g::Graph)
    s, t = st(g)
    while true
        path = residualpath(g, s, t)
        if path == nothing
            break
        end
        delta = residualflow(path)
        updateflow!(path, delta)
    end
    g
end

maxflow! (generic function with 1 method)

In [17]:
g = g4()
maxflow!(g)
g

G(V,E) = [4,5]
1: (1)-[20,20,0]->(2)
2: (1)-[10,10,0]->(3)
3: (2)-[10,10,0]->(4)
4: (3)-[20,20,0]->(4)
5: (2)-[10,30,0]->(3)


In [18]:
function makest!(g::Graph)
    sf, tf = 0, 0
    for v in g.vertices
        if v.flow > 0
            sf += v.flow
        elseif v.flow < 0
            tf += v.flow
        end
    end
    sf != 0 || error("Missing Sources")
    tf != 0 || error("Missing Targets")
    n = order(g)
    s = Vertex(n + 1, flow=sf)
    t = Vertex(n + 2, flow=tf)
    for v in g.vertices
        if v.flow > 0
            e = Edge(s, v, cap=v.flow)
            push!(g.edges, e)
        elseif v.flow < 0
            e = Edge(v, t, cap=-v.flow)
            push!(g.edges, e)
        end
        v.flow = 0
    end
    push!(g.vertices, s, t)
    g
end

makest! (generic function with 1 method)

In [19]:
s6load() = netg("../data/stndrd6.net")
s6 = s6load()
makest!(s6)
st(s6)

(301,302)

In [20]:
function resetst!(g::Graph)
    n = order(g) - 2
    s = g.vertices[n + 1]
    t = g.vertices[n + 2]
    m = size(g) - outdegree(s) - indegree(t)
    
    for e in s.edgesOut
        v = e.head
        pop!(v.edgesIn)
        v.flow = e.cap
    end
    for e in t.edgesIn
        v = e.tail
        pop!(v.edgesOut)
        v.flow = -e.cap
    end
    
    resize!(g.vertices, n)
    resize!(g.edges, m)
    g
end

resetst! (generic function with 1 method)

In [21]:
resetst!(s6)
s6

G(V,E) = [300,3174]
1: (1)-[0,150000,2355]->(232)
2: (1)-[0,150000,1252]->(223)
3: (1)-[0,150000,6180]->(212)
4: (1)-[0,150000,9177]->(166)
5: (1)-[0,150000,6102]->(157)
6: (1)-[0,150000,683]->(156)
7: (1)-[0,150000,6530]->(216)
8: (2)-[0,150000,5734]->(176)
9: (2)-[0,150000,5935]->(235)
10: (2)-[0,150000,1857]->(277)
... 3154 ...
3165: (150)-[0,150000,7363]->(248)
3166: (150)-[0,150000,5721]->(249)
3167: (150)-[0,150000,5167]->(251)
3168: (150)-[0,150000,7537]->(264)
3169: (150)-[0,150000,8172]->(267)
3170: (150)-[0,150000,6165]->(270)
3171: (150)-[0,150000,8544]->(274)
3172: (150)-[0,150000,3934]->(284)
3173: (150)-[0,150000,7686]->(295)
3174: (150)-[0,150000,8101]->(298)


In [22]:
makest!(s6)
maxflow!(s6)
resetst!(s6)
s6

G(V,E) = [300,3174]
1: (1)-[0,150000,2355]->(232)
2: (1)-[286,150000,1252]->(223)
3: (1)-[0,150000,6180]->(212)
4: (1)-[0,150000,9177]->(166)
5: (1)-[0,150000,6102]->(157)
6: (1)-[0,150000,683]->(156)
7: (1)-[0,150000,6530]->(216)
8: (2)-[1035,150000,5734]->(176)
9: (2)-[1403,150000,5935]->(235)
10: (2)-[0,150000,1857]->(277)
... 3154 ...
3165: (150)-[0,150000,7363]->(248)
3166: (150)-[0,150000,5721]->(249)
3167: (150)-[0,150000,5167]->(251)
3168: (150)-[78,150000,7537]->(264)
3169: (150)-[0,150000,8172]->(267)
3170: (150)-[0,150000,6165]->(270)
3171: (150)-[0,150000,8544]->(274)
3172: (150)-[0,150000,3934]->(284)
3173: (150)-[0,150000,7686]->(295)
3174: (150)-[0,150000,8101]->(298)


In [23]:
function isbalanced(g::Graph)
    sum([v.flow for v in g.vertices]) == 0
end

function hasconservation(g::Graph)
    for v in g.vertices
        flow(v) == 0 || return false
    end
    true
end

function hasnonnegativecapacity(g::Graph)
    for e in g.edges
        e.cap >= 0 || return false
    end
    true
end

function issingleedge(g::Graph)
    for v in g.vertices
        for e1 in v.edgesOut
            for e2 in v.edgesOut
                if !is(e1, e2) && is(e1.head, e2.head)
                    return false
                end
            end
        end
    end
    true
end

issingleedge (generic function with 1 method)

In [24]:
isbalanced(s6)

true

In [25]:
hasconservation(s6)

true

In [26]:
hasnonnegativecapacity(s6)

true

In [27]:
issingleedge(s6)

true

In [28]:
function isfeasibleflow(g::Graph)
    hasconservation(g) || return false
    for e in g.edges
        e.flow >= e.low && e.flow <= e.cap || return false
    end
    true
end

isfeasibleflow (generic function with 1 method)

In [29]:
isfeasibleflow(s6)

true

In [30]:
function g6()
    a = Vertex(1)
    b = Vertex(2)
    c = Vertex(3)
    d = Vertex(4)
    e = Vertex(5)
    t = Vertex(6)
    vrts = Vertex[a,b,c,d,e,t]
    
    e1 = Edge(a, b, cost=-4)
    e2 = Edge(a, t, cost=-3)
    e3 = Edge(b, d, cost=-1)
    e4 = Edge(b, e, cost=-2)
    e5 = Edge(c, b, cost=8)
    e6 = Edge(c, t, cost=3)
    e7 = Edge(d, a, cost=6)
    e8 = Edge(d, t, cost=4)
    e9 = Edge(e, c, cost=-3)
    e10 = Edge(e, t, cost=2)

    edgs = Edge[e1,e2,e3,e4,e5,e6,e7,e8,e9,e10]
    
    Graph(vrts,edgs)
end

g6 (generic function with 1 method)

In [31]:
g = g6()

G(V,E) = [6,10]
1: (1)-[0,0,-4]->(2)
2: (1)-[0,0,-3]->(6)
3: (2)-[0,0,-1]->(4)
4: (2)-[0,0,-2]->(5)
5: (3)-[0,0,8]->(2)
6: (3)-[0,0,3]->(6)
7: (4)-[0,0,6]->(1)
8: (4)-[0,0,4]->(6)
9: (5)-[0,0,-3]->(3)
10: (5)-[0,0,2]->(6)


In [32]:
function shortestpath(g::Graph, s::Vertex, t::Vertex)
    n = order(g)
    m = Array(Float32, n, n)
    m[1,:] = Inf32
    m[1,t.id] = 0.
    for i=2:n
        for v in g.vertices
            m[i,v.id] = if isempty(v.edgesOut)
                m[i-1,v.id]
            else
                w = Float32[m[i - 1, e.head.id] + e.cost for e in v.edgesOut]
                min(m[i-1,v.id],w...)
            end
        end
    end
    m
end

shortestpath (generic function with 1 method)

In [33]:
d = g.vertices[4]
t = g.vertices[6]
shortestpath(g,d,t)'

6x6 Array{Float32,2}:
 Inf     -3.0  -3.0  -4.0  -6.0  -6.0
 Inf    Inf     0.0  -2.0  -2.0  -2.0
 Inf      3.0   3.0   3.0   3.0   3.0
 Inf      4.0   3.0   3.0   2.0   0.0
 Inf      2.0   0.0   0.0   0.0   0.0
   0.0    0.0   0.0   0.0   0.0   0.0

In [34]:
function shortestpath(g::Graph, s::Vertex, t::Vertex)
    n = order(g)
    m = fill(Inf32, n)
    m[t.id] = 0.
    for i=2:n
        for v in g.vertices
            !isempty(v.edgesOut) || continue
            w = Float32[m[e.head.id] + e.cost for e in v.edgesOut]
            m[v.id] = min(m[v.id],w...)
        end
    end
    m
end

shortestpath (generic function with 1 method)

In [35]:
d = g.vertices[4]
t = g.vertices[6]
shortestpath(g,d,t)

6-element Array{Float32,1}:
 -6.0
 -2.0
  3.0
  0.0
  0.0
  0.0

In [36]:
function shortestpath(g::Graph, s::Vertex, t::Vertex)
    n = order(g)
    m = fill(Inf, n)
    m[t.id] = 0.
    p = Array(Edge, n)
    p[t.id] = Edge(t,Vertex(0),0,0,0,0)
    for i=2:n
        for v in g.vertices
            !isempty(v.edgesOut) || continue
            mmin = m[v.id]
            k = nothing #Edge
            for e in v.edgesOut
                mk = m[e.head.id] + e.cost
                if mk < mmin
                    mmin = mk
                    k = e
                end
            end
            if k != nothing
                p[v.id] = k
                m[v.id] = mmin
            end
        end
    end
    
    r, i = Array(Edge, n), 0
    e = p[s.id]
    while e.head.id != 0
        r[i += 1] = e
        e = p[e.head.id]
    end
    resize!(r, i)
    r, round(Int, m[s.id])
end

shortestpath (generic function with 1 method)

In [37]:
d = g.vertices[1]
t = g.vertices[6]
shortestpath(g,d,t)

([(1)-[0,0,-4]->(2),(2)-[0,0,-2]->(5),(5)-[0,0,-3]->(3),(3)-[0,0,3]->(6)],-6)

In [38]:
b = g.vertices[2]
c = g.vertices[3]
shortestpath(g,b,c)

([(2)-[0,0,-2]->(5),(5)-[0,0,-3]->(3)],-5)

In [39]:
function g4neg()
    v1 = Vertex(1, flow=4)
    v2 = Vertex(2)
    v3 = Vertex(3)
    v4 = Vertex(4, flow=-4)
    vrts = Vertex[v1, v2, v3, v4]
    
    e1 = Edge(v1, v2, cost=2,cap=4)
    e2 = Edge(v1, v3, cost=2,cap=2)
    e3 = Edge(v2, v3, cost=1,cap=2)
    e4 = Edge(v2, v4, cost=3,cap=3)
    e5 = Edge(v3, v4, cost=1,cap=5)
    edgs = Edge[e1, e2, e3, e4, e5]
    
    Graph(vrts, edgs)
end

g4neg (generic function with 1 method)

In [40]:
g = g4neg()

G(V,E) = [4,5]
1: (1)-[0,4,2]->(2)
2: (1)-[0,2,2]->(3)
3: (2)-[0,2,1]->(3)
4: (2)-[0,3,3]->(4)
5: (3)-[0,5,1]->(4)


In [41]:
maxflow!(g)

G(V,E) = [4,5]
1: (1)-[3,4,2]->(2)
2: (1)-[1,2,2]->(3)
3: (2)-[0,2,1]->(3)
4: (2)-[3,3,3]->(4)
5: (3)-[1,5,1]->(4)


In [42]:
isfeasibleflow(g)

true

In [43]:
function findcycle(p::Array{Edge,1}, i::Int)
    mark = falses(length(p))
    j = i
    while j != 0 && !mark[j]
        mark[j] = true
        j = p[j].head.id
    end
    j
end

function cycle(p::Array{Edge,1}, i::Int)
    r = Array(Edge, length(p))
    n = 0
    j = i
    while n == 0 || i != j
        r[n += 1] = p[j]
        j = p[j].head.id
    end
    resize!(r, n)
    r
end

cycle (generic function with 1 method)

In [44]:
function pcycle()
    v1 = Vertex(1)
    v2 = Vertex(2)
    v3 = Vertex(3)
    v4 = Vertex(4)
    e1 = Edge(v1, v2)
    e2 = Edge(v2, v3)
    e3 = Edge(v3, v2)
    e4 = Edge(v3, v4)
    Edge[e1,e2,e3,e4]
end

pcycle (generic function with 1 method)

In [45]:
pc = pcycle()

4-element Array{Edge,1}:
 (1)-[0,0,0]->(2)
 (2)-[0,0,0]->(3)
 (3)-[0,0,0]->(2)
 (3)-[0,0,0]->(4)

In [46]:
cycle(pc, 2)

2-element Array{Edge,1}:
 (2)-[0,0,0]->(3)
 (3)-[0,0,0]->(2)

In [47]:
cycle(pc, 3)

2-element Array{Edge,1}:
 (3)-[0,0,0]->(2)
 (2)-[0,0,0]->(3)

In [48]:
findcycle(pc, 1)

2

In [49]:
function negativecycle(g::Graph, s::Vertex, t::Vertex)
    n = order(g)
    m = fill(Inf, n)
    m[t.id] = 0.
    p = Array(Edge, n)
    p[t.id] = Edge(t,Vertex(0),0,0,0,0)
    for _=2:n
        for v in g.vertices
            !isempty(v.edgesOut) || continue
            mmin = m[v.id]
            k = nothing #Edge
            for e in v.edgesOut
                mk = m[e.head.id] + e.cost
                if mk < mmin
                    mmin = mk
                    k = e
                end
            end
            if k != nothing
                p[v.id] = k
                m[v.id] = mmin
                
                i = findcycle(p, v.id)
                i == 0 || return cycle(p, i)
            end
        end
    end
    nothing
end

negativecycle (generic function with 1 method)

In [50]:
function g4negt()
    v1 = Vertex(1)
    v2 = Vertex(2)
    v3 = Vertex(3)
    v4 = Vertex(4)
    V = Vertex[v1,v2,v3,v4]
    e1 = Edge(v1, v2, cost=1)
    e2 = Edge(v2, v3, cost=2)
    e3 = Edge(v3, v2, cost=-3)
    e4 = Edge(v3, v4, cost=3)
    E = Edge[e1,e2,e3,e4]
    Graph(V, E)
end

g4negt (generic function with 1 method)

In [51]:
g = g4negt()

G(V,E) = [4,4]
1: (1)-[0,0,1]->(2)
2: (2)-[0,0,2]->(3)
3: (3)-[0,0,-3]->(2)
4: (3)-[0,0,3]->(4)


In [52]:
s = g.vertices[1]
t = g.vertices[4]
negativecycle(g, s, t)

2-element Array{Edge,1}:
 (3)-[0,0,-3]->(2)
 (2)-[0,0,2]->(3) 

In [53]:
# Reference
# Algorithms Design. Tardos and Kleinberg
# 6.8 Shortest Paths in a Graph, pp. 290-297
# (Bellman-Ford algorithm)
function negativecycle(g::Graph)
    n = order(g)
    
    m = fill(Inf, n)
    p = Array(PathNode, n)
    for v in g.vertices
        v.flow < 0 || continue
        m[v.id] = 0.
        p[v.id] = PathNode(Edge(v,Vertex(0),0,0,0,0))
    end
    
    mark = Array(Bool,n)
    _findcycle(p::Array{PathNode,1}, i::Int) = findcycle(mark,p,i)
    
    for _=2:n
        for v in g.vertices
            mmin = m[v.id]
            k = nothing #Edge
            reverse = false
            for e in v.edgesOut
                e.flow < e.cap || continue
                mk = m[e.head.id] + e.cost
                if mk < mmin
                    mmin = mk
                    k = e
                end
            end
            for e in v.edgesIn
                e.flow > 0 || continue
                mk = m[e.tail.id] - e.cost
                if mk < mmin
                    mmin = mk
                    k = e
                    reverse = true
                end
            end
            if k != nothing
                p[v.id] = PathNode(k,reverse)
                m[v.id] = mmin
                
                i = _findcycle(p, v.id)
                i == 0 || return cycle(p, i)
            end
        end
    end
    nothing
end

function findcycle(mark::Array{Bool,1}, p::Array{PathNode,1}, i::Int)
    mark[:] = false
    j = i
    while j != 0 && !mark[j]
        mark[j] = true
        j = head(p[j]).id
    end
    j
end

function cycle(p::Array{PathNode,1}, i::Int)
    r = Array(PathNode, length(p))
    n = 0
    j = i
    while n == 0 || i != j
        r[n += 1] = p[j]
        j = head(p[j]).id
    end
    resize!(r, n)
    r
end

cycle (generic function with 2 methods)

In [54]:
g = g4neg()
maxflow!(g)
isfeasibleflow(g)

true

In [55]:
c = negativecycle(g)

4-element Array{PathNode,1}:
 (4)-[0,3,-3]->(2), reverse=true
 (2)-[1,4,-2]->(1), reverse=true
 (1)-[1,2,2]->(3), reverse=false
 (3)-[1,5,1]->(4), reverse=false

In [56]:
d = availableflow(c)

1

In [57]:
updateflow!(c, d)
c

4-element Array{PathNode,1}:
 (4)-[1,3,-3]->(2), reverse=true
 (2)-[2,4,-2]->(1), reverse=true
 (1)-[2,2,2]->(3), reverse=false
 (3)-[2,5,1]->(4), reverse=false

In [58]:
g

G(V,E) = [4,5]
1: (1)-[2,4,2]->(2)
2: (1)-[2,2,2]->(3)
3: (2)-[0,2,1]->(3)
4: (2)-[2,3,3]->(4)
5: (3)-[2,5,1]->(4)


In [59]:
isfeasibleflow(g)

true

In [60]:
c = negativecycle(g)

3-element Array{PathNode,1}:
 (4)-[1,3,-3]->(2), reverse=true
 (2)-[0,2,1]->(3), reverse=false
 (3)-[2,5,1]->(4), reverse=false

In [61]:
d = availableflow(c)

2

In [62]:
updateflow!(c, d)
g

G(V,E) = [4,5]
1: (1)-[2,4,2]->(2)
2: (1)-[2,2,2]->(3)
3: (2)-[2,2,1]->(3)
4: (2)-[0,3,3]->(4)
5: (3)-[4,5,1]->(4)


In [63]:
isfeasibleflow(g)

true

In [64]:
negativecycle(g) == nothing

true

In [65]:
cost(g::Graph) =  sum([e.flow * e.cost for e in g.edges])

cost (generic function with 2 methods)

In [66]:
cost(g)

14

In [67]:
for e in g.edges
    e.flow = 0
end
maxflow!(g)

G(V,E) = [4,5]
1: (1)-[3,4,2]->(2)
2: (1)-[1,2,2]->(3)
3: (2)-[0,2,1]->(3)
4: (2)-[3,3,3]->(4)
5: (3)-[1,5,1]->(4)


In [68]:
cost(g)

18

In [69]:
# Reference
# Network Flows. Ahuja, Magnanti and Orlin
# 9.6 Cycle-Canceling algorithm and the integrality property, pp. 317-320
function mincost_cyclecanceling!(g::Graph)
    isfeasibleflow(g) || error("missing flow")
    while true
        cycle = negativecycle(g)
        if cycle == nothing
            break
        end
        delta = availableflow(cycle)
        updateflow!(cycle, delta)
    end
    g
end 

mincost_cyclecanceling! (generic function with 1 method)

In [70]:
g = g4neg()
maxflow!(g)
mincost_cyclecanceling!(g)

G(V,E) = [4,5]
1: (1)-[2,4,2]->(2)
2: (1)-[2,2,2]->(3)
3: (2)-[2,2,1]->(3)
4: (2)-[0,3,3]->(4)
5: (3)-[4,5,1]->(4)


In [71]:
isfeasibleflow(g)

true

In [72]:
cost(g)

14

In [73]:
s6 = s6load()
makest!(s6)
maxflow!(s6)
resetst!(s6)
isfeasibleflow(s6) || error("WAT!?")
cost(s6)

711413602

In [74]:
mincost_cyclecanceling!(s6)
isfeasibleflow(s6) || error("WAT!?")
cost(s6)

191968577